In [1]:
import matplotlib
matplotlib.use('Qt5Agg')
from matplotlib import pyplot as plt
plt.rcParams.update({'font.size': 22})
import numpy as np
import sys
sys.path.insert(0, '/home/misa/APDFT/prototyping/atomic_energies/')
from parse_cube_files import CUBE

# dsgdb9nsd_002626

In [2]:
ve38 = CUBE(r'/home/misa/APDFT/prototyping/atomic_energies/results/test_calculations/lambda_integrals/dsgdb9nsd_002626/DENS_38.cube')
ve30 = CUBE(r'/home/misa/APDFT/prototyping/atomic_energies/results/test_calculations/lambda_integrals/dsgdb9nsd_002626/DENS_30.cube')
ve23 = CUBE(r'/home/misa/APDFT/prototyping/atomic_energies/results/test_calculations/lambda_integrals/dsgdb9nsd_002626/DENS_23.cube')
ve15 = CUBE(r'/home/misa/APDFT/prototyping/atomic_energies/results/test_calculations/lambda_integrals/dsgdb9nsd_002626/DENS_15.cube')
ve8 = CUBE(r'/home/misa/APDFT/prototyping/atomic_energies/results/test_calculations/lambda_integrals/dsgdb9nsd_002626/DENS_8.cube')
ve0 = CUBE(r'/home/misa/APDFT/prototyping/atomic_energies/results/test_calculations/lambda_integrals/free_electron_gas.cube')

In [18]:
from alchemy_tools import atomic_energy_decomposition
p38 = '/home/misa/APDFT/prototyping/atomic_energies/results/test_calculations/lambda_integrals/dsgdb9nsd_002626/DENS_38.cube'
p30 = '/home/misa/APDFT/prototyping/atomic_energies/results/test_calculations/lambda_integrals/dsgdb9nsd_002626/DENS_30.cube'
p23 = '/home/misa/APDFT/prototyping/atomic_energies/results/test_calculations/lambda_integrals/dsgdb9nsd_002626/DENS_23.cube'
p15 = '/home/misa/APDFT/prototyping/atomic_energies/results/test_calculations/lambda_integrals/dsgdb9nsd_002626/DENS_15.cube'
p8 = '/home/misa/APDFT/prototyping/atomic_energies/results/test_calculations/lambda_integrals/dsgdb9nsd_002626/DENS_8.cube'
p0 = '/home/misa/APDFT/prototyping/atomic_energies/results/test_calculations/lambda_integrals/free_electron_gas.cube'
cubes = [(p0, 0), (p8, 8/38), (p15, 15/38), (p23, 23/38), (p30, 30/38), (p38, 1)]
# ae, alchpots = atomic_energy_decomposition(cubes)
atomic_energy_decomposition(cubes, '/home/misa/APDFT/prototyping/atomic_energies/results/test_calculations/lambda_integrals/output')

In [4]:
print(ae)

[-52.41941963 -38.81593713 -39.49926896 -39.88880231 -38.57914187
 -36.29823107 -32.4020716   -5.39594217  -5.70814393  -5.64600143
  -5.78079931  -5.79580329  -5.5379296   -5.53118386  -4.54357057]


# Integration

In [5]:
from alchemy_tools import integrate_lambda_density, calculate_atomic_energies, calculate_alchemical_potential

In [6]:
lambda_densities = [ve0, ve8, ve15, ve23, ve30, ve38]
density_arrays = []
for dens in lambda_densities:
    dens.scale()
    density_arrays.append(dens.data_scaled)
lam_vals = [0, 8/38, 15/38, 23/38, 30/38, 1]

In [7]:
av_dens = integrate_lambda_density(density_arrays, lam_vals)

# Plots

In [8]:
# projection on line
x = np.linspace(0, 20, 175)
sa = (1,2)
av_dens_1D = av_dens.sum(axis=sa)
pd_1D = [ve38.project(sa), ve30.project(sa), ve23.project(sa), ve15.project(sa), ve8.project(sa), ve0.project(sa), av_dens_1D]
label = [r'$\rho(\lambda = 1.0)$',r'$\rho(\lambda \approx 0.79)$', r'$\rho(\lambda \approx 0.61)$', r'$\rho(\lambda \approx 0.39)$', r'$\rho(\lambda \approx 0.21)$', r'$\rho(\lambda = 0)$', r'$\tilde{\rho} \approx \int_0^1 d\lambda \rho(\lambda, r)$']
fig, ax = plt.subplots(1,1)
for idx, pd in enumerate(pd_1D):
    # bigger linewidth for lambda-averaged density
    if idx == len(pd_1D)-1:
        ax.plot(x, pd, '--', label=label[idx])
    else:
        ax.plot(x, pd, label=label[idx])

ax.set_title('Boxsize = 20 Ang')
ax.set_xlabel('Cell coordinate (Ang)')
ax.set_xlim(0.5, 19.5)
ax.set_ylabel(r'Projected Density $\rho(x_0)$ (Bohr$^{-3}$) ')

ax.legend()

# Alchemical Potential and Atomic Energies

In [9]:
nuclei = ve38.atoms
meshgrid = ve38.get_grid()
meshgrid_xyz = np.vstack([_.flatten() for _ in meshgrid]).T

In [10]:
nuc=np.array(ve38.atoms)[0][1:].T

In [11]:
np.array(ve38.atoms)[0][1:4]

array([14.673877, 14.892382, 22.556505])

In [12]:
calculate_alchemical_potential(av_dens, meshgrid, nuc)

-6.552427453817083

In [13]:
# %timeit atomic_energies = get_atomic_energies(av_dens, nuclei, meshgrid)
atomic_energies = calculate_atomic_energies(av_dens, nuclei, meshgrid)

In [14]:
# %timeit dist_gpt_nuc = np.linalg.norm(meshgrid_xyz - nuclei[0][1:], axis=1)
#(density.flatten()/dist_gpt_nuc).sum()


In [15]:
atomic_energies

(array([-52.41941963, -38.81593713, -39.49926896, -39.88880231,
        -38.57914187, -36.29823107, -32.4020716 ,  -5.39594217,
         -5.70814393,  -5.64600143,  -5.78079931,  -5.79580329,
         -5.5379296 ,  -5.53118386,  -4.54357057]),
 array([-6.55242745, -6.46932286, -6.58321149, -6.64813372, -6.42985698,
        -6.04970518, -5.40034527, -5.39594217, -5.70814393, -5.64600143,
        -5.78079931, -5.79580329, -5.5379296 , -5.53118386, -4.54357057]))